### Packages

In [17]:
import sys
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.models import load_model
from keras_metrics import KerasMetrics
from keras.callbacks import EarlyStopping
from keras import optimizers
import numpy as np
import random
import pickle
import decoding2
import coding_function as cf
import decoding_function as df
import csv
from Files import nodes
import os
sys.path.append('../')

In [4]:
"""
Path Sugestions:
Files/training_data_set.csv
Files/biased_train1.csv
Files/biased_train2.csv
Files/prop_55.csv
"""
def load_data(path):
    import pandas as pd
    with open(path,'r') as csvfile:
        readcsv = csv.reader(csvfile, delimiter = ',')
        phoneticinf = []
        phoneticI = []
        for row in readcsv:
            phoneticinf.append(row[1])
            phoneticI.append(row[3])
        cf.dataTest(phoneticinf,phoneticI) #tests if dataset is ok
        vec_inf = []
        vec_I = []
        for item in phoneticinf:
            vec_inf.append(cf.coding(item))
        for item in phoneticI:
            vec_I.append(cf.coding(item))   
    d = {'infinitivo': phoneticinf, 'vec_inf': vec_inf, 'p pessoa': phoneticI, 'vec_I': vec_I}
    df = pd.DataFrame(data=d)
    X = np.array(df['vec_inf'].tolist())
    Y = np.array(df['vec_I'].tolist())
    #X = np.array(list(dictioinf.values())) #input column
    #Y = np.array(list(dictioI.values())) #output column
    return X,Y

def pipeline(verbs):
    """
    Pipeline receives a list of verbs and decodes it.
    
    :verbs type: list
    :r type: string
    """
    test_list = []
    for i in verbs:
        coding = cf.coding(i)        
        test_list.append(coding)
    test_list = np.array(test_list)
    prediction = model.predict(test_list)
    for i in prediction:
        print(decoding2.decoding(i))
    return test_list.shape

### 1. Load Data

In [5]:
#path = '../data/prop_55.csv' #len 464
path = '../data/prop_65.csv'
#path = '../data/prop_75.csv'
#path = '../data/prop_85.csv'
#path = '../data/prop_95.csv'
X,Y = load_data(path)
len(X)

579

### 2. Fit Neural Net (Optional)

In [31]:
# 1. Define Model
model = Sequential()
model.add(Dense(460, input_shape=(460,), activation='sigmoid'))


# 2. Compile model
keras_metrics = KerasMetrics()
f1_score = keras_metrics.fbeta_score
model.compile( 
    optimizer='adam', 
    loss='mean_squared_error',
    metrics = [f1_score, 
                           keras_metrics.fbeta_score,
                           keras_metrics.recall,
                          keras_metrics.precision]
            )

# 3. Fit model
stopper = EarlyStopping(monitor='fbeta_score_1', min_delta=0.00005, patience=50, verbose=1, mode='max')
model.fit(X,Y,epochs = 150, batch_size=600,verbose=False,callbacks= [stopper])

# 4. Evaluate model
scores = model.evaluate(X, Y)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

# 5. Save model
#model.save('../data/f1_test')

Epoch 1/150
579/579 [==============================] - 0s - loss: 0.2605 - fbeta_score_1: 0.2898 - fbeta_score_2: 0.2898 - recall: 0.4924 - precision: 0.2054
Epoch 2/150
579/579 [==============================] - 0s - loss: 0.2507 - fbeta_score_1: 0.2967 - fbeta_score_2: 0.2967 - recall: 0.4702 - precision: 0.2167
Epoch 3/150
579/579 [==============================] - 0s - loss: 0.2382 - fbeta_score_1: 0.3066 - fbeta_score_2: 0.3066 - recall: 0.4408 - precision: 0.2351
Epoch 4/150
579/579 [==============================] - 0s - loss: 0.2265 - fbeta_score_1: 0.3189 - fbeta_score_2: 0.3189 - recall: 0.4156 - precision: 0.2587
Epoch 5/150
579/579 [==============================] - 0s - loss: 0.2156 - fbeta_score_1: 0.3318 - fbeta_score_2: 0.3318 - recall: 0.3923 - precision: 0.2875
Epoch 6/150
579/579 [==============================] - 0s - loss: 0.2055 - fbeta_score_1: 0.3470 - fbeta_score_2: 0.3470 - recall: 0.3744 - precision: 0.3234
Epoch 7/150
579/579 [==============================]

### 2. (Or Load Trained Model)

In [4]:
model = load_model('../data/prop_55_batch_353')
#model = load_model('../data/biased_model_test1')

#### 2.1 Train existing model

In [46]:
model = load_model('../data/prop_65_batch_579')
model.fit(X,Y, epochs=400, batch_size=579,verbose=False)
model.save('../data/prop_65_batch_579')

### 3. Check Results
pipeline(['#para#','#fexa#','#mexe#','#konta#','#testa#','#abri#','#kala#','#fala#'])

|       	|       	| front 	| middle  	| back    	|
|:-----:	|-------	|------:	|---------	|---------	|
|       	|       	| v/u   	| v/u     	| v/u     	|
|  **int**  | **stop**  | b/p   	| d/t     	| g/k     	|
|       	| **nasal** | m/-   	| n/-     	| -       	|
| **cont**  | **fric**  | v/f   	| z/s    	| j/x   	|
|       	| **liq**   | l/-   	| r/-     	| -/h     	|
| **vowel** | **high**  | e/i   	| -       	| o/u     	|
|       	| **low**   | -/E   	| -/a     	| -/O     	|

## Results:
|     Model Name     | Test File     | Total Irregular Verbs | Total Verbs | Proportion | Epochs | Batch Size |
|:------------------:|---------------|----------------------:|-------------|------------|--------|-------------|
| biased_model_test1 | biased_train_1 |                    66 | 139         | 47%        | 600    |  20          |

In [45]:
model = load_model('../data/biased_model_test1')
pipeline(['#para#','#fexa#','#mexe#','#konta#','#testa#','#abri#','#kala#','#fala#'])

#pa
Decodificado Loop: #paru, Final antes do Loop: ru#
#fe
Decodificado Loop: #feku#, Final antes do Loop: xu#
#me
Decodificado Loop: #mexu, Final antes do Loop: xu#
#ko
Decodificado Loop: #ko#, Final antes do Loop: tu#
#tE
Decodificado Loop: #tEspu#, Final antes do Loop: tu#
#ag
Decodificado Loop: #agva#, Final antes do Loop: zu#
#pa
Decodificado Loop: #palu, Final antes do Loop: lu#
#fa
Decodificado Loop: #falu, Final antes do Loop: lu#


(8, 460)

|     Model Name     | Test File     | Total Irregular Verbs | Total Verbs | Proportion | Epochs | Batch Sizes |
|:------------------:|---------------|----------------------:|-------------|------------|--------|-------------|
| unbiased_model | training_data_set |          151          |    410      |     37%    | 600    |  20          |

In [78]:
model = load_model('../data/unbiased_model')
pipeline(['#para#','#fexa#','#mexe#','#abri#','#kala#','#fala#','#teka#'])

#pa
Decodificado Loop: #pari#, Final antes do Loop: ru#
#fe
Decodificado Loop: #fexu, Final antes do Loop: xu#
#me
Decodificado Loop: #mexu, Final antes do Loop: xu#
#ab
Decodificado Loop: #abru, Final antes do Loop: ru#
#ka
Decodificado Loop: #kalu, Final antes do Loop: lu#
#fa
Decodificado Loop: #falu, Final antes do Loop: lu#
#te
Decodificado Loop: #teku, Final antes do Loop: ku#


(7, 460)

|     Model Name     | Test File     | Total Irregular Verbs | Total Verbs | Proportion | Epochs | Batch Size  |
|:------------------:|---------------|----------------------:|-------------|------------|--------|-------------|
| biased_model_test2 | biased_train2 |        88             |  132        |    66%     | 600    |  20         |

In [4]:
model = load_model('../data/biased_model_test2')

In [21]:
#pipeline(['#fexa#','#mexe#','#konta#','#testa#','#kala#','#fala#',"#bota#"])

|     Model Name     | Test File     | Total Irregular Verbs | Total Verbs | Proportion | Epochs | Batch Size  |
|:------------------:|---------------|----------------------:|-------------|------------|--------|-------------|
| biased_model_test3 | biased_train3 |        88             |  156        |    50%     | 600    |  20         |

In [5]:
model = load_model('../data/biased_model_test3')

In [6]:
pipeline(['#para#','#fexa#','#mexe#','#konta#','#testa#','#kala#','#fala#'])

#pa
Decodificado Loop: #paru, Final antes do Loop: ru#
#fi
Decodificado Loop: #fiku#, Final antes do Loop: xu#
#me
Decodificado Loop: #mesu, Final antes do Loop: su#
#ko
Decodificado Loop: #ko#, Final antes do Loop: tu#
#dE
Decodificado Loop: #dE#, Final antes do Loop: tu#
#pa
Decodificado Loop: #palu, Final antes do Loop: lu#
#fa
Decodificado Loop: #falu, Final antes do Loop: lu#


(7, 460)

|     Model Name     | Test File     | Total Irregular Verbs | Total Verbs | Proportion | Epochs | Batch Size  |
|:------------------:|---------------|----------------------:|-------------|------------|--------|-------------|
| biased_model_test2 | biased_train2 |        88             |  132        |    66%     | 1000    |  20         |

In [20]:
#pipeline(['#para#','#fexa#','#mexe#','#abri#','#kala#','#fala#','#teka#'])

## Proporcao 55% Irregulares

|     Model Name     | Test File     | Total Irregular Verbs | Total Verbs | Proportion | Epochs | Batch Size  |
|:------------------:|---------------|----------------------:|-------------|------------|--------|-------------|
| prop_55_batch_400           | prop_55       |        209            |  464        |    55%     | 1000    |  400         |

In [238]:
path = '../data/prop_55.csv'
X,Y = load_data(path)
len(X)

464

In [72]:
model = load_model('../data/prop_55_batch_400')
pipeline(['#pega#','#sega#','#seka#','#leva#'])

#pe
Decodificado Loop: #pe, Final antes do Loop: gu#
#se
Decodificado Loop: #se, Final antes do Loop: gu#
#se
Decodificado Loop: #se, Final antes do Loop: ku#
#le
Decodificado Loop: #le, Final antes do Loop: vu#


(4, 460)

In [17]:
model = load_model('../data/prop_55_batch_400')
pipeline(['#ora#',"#mora#",'#joga#','#posta#'])

#er
Decodificado Loop: #er, Final antes do Loop: ru#
#bi
Decodificado Loop: #bi, Final antes do Loop: ru#
#xo
Decodificado Loop: #xo, Final antes do Loop: gu#
#ai
Decodificado Loop: #ai, Final antes do Loop: tu#


(4, 460)

In [21]:
model = load_model('../data/prop_55_batch_400')
pipeline(['#sortia#','#hemedia#'])

#su
Decodificado Loop: #su, Final antes do Loop: iu#
#xe
Decodificado Loop: #xe, Final antes do Loop: iu#


(2, 460)

In [23]:
model = load_model('../data/prop_55_batch_400')
pipeline(['#vense#','#proteje#','#eskolie#'])

#be
Decodificado Loop: #be, Final antes do Loop: tu#
#pr
Decodificado Loop: #pr, Final antes do Loop: ju#
#bz
Decodificado Loop: #bz, Final antes do Loop: fu#


(3, 460)

In [24]:
model = load_model('../data/prop_55_batch_400')
pipeline(['#po#','#hepo#'])

#po
Decodificado Loop: #po, Final antes do Loop: iu#
#xe
Decodificado Loop: #xe, Final antes do Loop: iu#


(2, 460)

In [25]:
model = load_model('../data/prop_55_batch_400')
pipeline(['#tendi#','#jenti#','#fenti#','#hendi#'])

#te
Decodificado Loop: #te, Final antes do Loop: du#
#je
Decodificado Loop: #je, Final antes do Loop: tu#
#fe
Decodificado Loop: #fe, Final antes do Loop: tu#
#xe
Decodificado Loop: #xe, Final antes do Loop: tu#


(4, 460)

In [26]:
model = load_model('../data/prop_55_batch_400')
pipeline(['#tosi#','#kobri#','#goti#'])

#te
Decodificado Loop: #te, Final antes do Loop: su#
#ko
Decodificado Loop: #ko, Final antes do Loop: ru#
#go
Decodificado Loop: #go, Final antes do Loop: tu#


(3, 460)

In [27]:
model = load_model('../data/prop_55_batch_400')
pipeline(['#nase#','#faze#'])

#da
Decodificado Loop: #da, Final antes do Loop: su#
#fa
Decodificado Loop: #fa, Final antes do Loop: zu#


(2, 460)

In [28]:
model = load_model('../data/prop_55_batch_400')
pipeline(['#mata#',"#paga#","#bate#","#sai#"])

#ma
Decodificado Loop: #ma, Final antes do Loop: tu#
#pa
Decodificado Loop: #pa, Final antes do Loop: gu#
#ba
Decodificado Loop: #ba, Final antes do Loop: tu#
#ta
Decodificado Loop: #ta, Final antes do Loop: iu#


(4, 460)

In [29]:
path = '../data/prop_55.csv'
X,Y = load_data(path)
model = load_model('../data/prop_55_batch_400')
model.fit(X,Y, epochs=400, batch_size=400,verbose=False)
model.save('../data/prop_55_batch_400_2')

|     Model Name     | Test File     | Total Irregular Verbs | Total Verbs | Proportion | Epochs | Batch Size  |
|:------------------:|---------------|----------------------:|-------------|------------|--------|-------------|
| prop_55_batch_400_2   | prop_55       |        209           |  464        |    55%     | 1400    |  400        |

In [71]:
model = load_model('../data/prop_55_batch_400_2')
pipeline(['#pega#','#sega#','#seka#','#leva#'])

#pe
Decodificado Loop: #pe, Final antes do Loop: gu#
#se
Decodificado Loop: #se, Final antes do Loop: gu#
#se
Decodificado Loop: #se, Final antes do Loop: ku#
#le
Decodificado Loop: #le, Final antes do Loop: vu#


(4, 460)

In [32]:
model = load_model('../data/prop_55_batch_400_2')
pipeline(['#ora#',"#mora#",'#joga#','#posta#','#hetoma#'])

#er
Decodificado Loop: #er, Final antes do Loop: ru#
#mi
Decodificado Loop: #mi, Final antes do Loop: ru#
#xo
Decodificado Loop: #xo, Final antes do Loop: gu#
#bu
Decodificado Loop: #bu, Final antes do Loop: tu#
#xe
Decodificado Loop: #xe, Final antes do Loop: bu#


(5, 460)

In [36]:
model = load_model('../data/prop_55_batch_400_2')
pipeline(['#sortia#'])

#su
Decodificado Loop: #su, Final antes do Loop: iu#


(1, 460)

In [37]:
model = load_model('../data/prop_55_batch_400_2')
pipeline(['#vense#'])

#be
Decodificado Loop: #be, Final antes do Loop: tu#


(1, 460)

In [38]:
model = load_model('../data/prop_55_batch_400_2')
pipeline(['#kompo#','#po#'])

#ko
Decodificado Loop: #ko, Final antes do Loop: iu#
#po
Decodificado Loop: #po, Final antes do Loop: iu#


(2, 460)

In [40]:
model = load_model('../data/prop_55_batch_400_2')
pipeline(['#jenti#','#fenti#','#hendi#','#tendi#'])

#je
Decodificado Loop: #je, Final antes do Loop: tu#
#fe
Decodificado Loop: #fe, Final antes do Loop: tu#
#xe
Decodificado Loop: #xe, Final antes do Loop: du#
#te
Decodificado Loop: #te, Final antes do Loop: du#


(4, 460)

In [41]:
model = load_model('../data/prop_55_batch_400_2')
pipeline(['#tosi#','#kobri#','#goti#'])#,'#fosi#']) 

#to
Decodificado Loop: #to, Final antes do Loop: su#
#ko
Decodificado Loop: #ko, Final antes do Loop: ru#
#go
Decodificado Loop: #go, Final antes do Loop: tu#


(3, 460)

In [42]:
model = load_model('../data/prop_55_batch_400_2')
pipeline(['#afaze#','#nase#'])

#ai
Decodificado Loop: #ai, Final antes do Loop: su#
#da
Decodificado Loop: #da, Final antes do Loop: su#


(2, 460)

In [43]:
model = load_model('../data/prop_55_batch_400_2')
pipeline(['#mata#',"#paga#",'#sai#','#bate#'])

#ma
Decodificado Loop: #ma, Final antes do Loop: tu#
#pa
Decodificado Loop: #pa, Final antes do Loop: gu#
#ta
Decodificado Loop: #ta, Final antes do Loop: iu#
#ba
Decodificado Loop: #ba, Final antes do Loop: tu#


(4, 460)

## Proporcao 65% Irregulares


|     Model Name     | Test File     | Total Irregular Verbs | Total Verbs | Proportion | Epochs | Batch Size  |
|:------------------:|---------------|----------------------:|-------------|------------|--------|-------------|
| prop_65_batch_100       | prop_65       |      209        |  579        |    65%     | 1000    |  100         |

In [62]:
model = load_model('../data/prop_65_batch_100')
pipeline(['#pega#','#sega#','#seka#','#testa#','#presta#'])

#pe
Decodificado Loop: #pe, Final antes do Loop: gu#
#se
Decodificado Loop: #se, Final antes do Loop: gu#
#si
Decodificado Loop: #si, Final antes do Loop: ku#
#de
Decodificado Loop: #de, Final antes do Loop: tu#
#pr
Decodificado Loop: #pr, Final antes do Loop: tu#


(5, 460)

In [63]:
model = load_model('../data/prop_65_batch_100')
pipeline(['#ora#',"#mora#",'#joga#','#aposta#','#hetoma#'])

#er
Decodificado Loop: #er, Final antes do Loop: ru#
#me
Decodificado Loop: #me, Final antes do Loop: ru#
#jo
Decodificado Loop: #jo, Final antes do Loop: gu#
#ap
Decodificado Loop: #ap, Final antes do Loop: tu#
#xe
Decodificado Loop: #xe, Final antes do Loop: bu#


(5, 460)

In [64]:
model = load_model('../data/prop_65_batch_100')
pipeline(['#sortia#','#media#'])

#su
Decodificado Loop: #su, Final antes do Loop: iu#
#me
Decodificado Loop: #me, Final antes do Loop: iu#


(2, 460)

In [65]:
model = load_model('../data/prop_65_batch_100')
pipeline(['#vense#','#proteje#'])

#ve
Decodificado Loop: #ve, Final antes do Loop: su#
#pr
Decodificado Loop: #pr, Final antes do Loop: ju#


(2, 460)

In [66]:
model = load_model('../data/prop_65_batch_100')
pipeline(['#po#','#hepo#'])

#po
Decodificado Loop: #po, Final antes do Loop: iu#
#xe
Decodificado Loop: #xe, Final antes do Loop: iu#


(2, 460)

In [67]:
model = load_model('../data/prop_65_batch_100')
pipeline(['#jenti#','#fenti#','#hendi#','#tendi#'])

#je
Decodificado Loop: #je, Final antes do Loop: tu#
#fe
Decodificado Loop: #fe, Final antes do Loop: tu#
#xe
Decodificado Loop: #xe, Final antes do Loop: tu#
#te
Decodificado Loop: #te, Final antes do Loop: tu#


(4, 460)

In [68]:
model = load_model('../data/prop_65_batch_100')
pipeline(['#tosi#','#kobri#','#goti#'])#'#fosi#']) 

#to
Decodificado Loop: #to, Final antes do Loop: su#
#ko
Decodificado Loop: #ko, Final antes do Loop: ru#
#go
Decodificado Loop: #go, Final antes do Loop: tu#


(3, 460)

In [69]:
model = load_model('../data/prop_65_batch_100')
pipeline(['#afaze#','#nase#'])

#as
Decodificado Loop: #as, Final antes do Loop: su#
#na
Decodificado Loop: #na, Final antes do Loop: su#


(2, 460)

In [70]:
model = load_model('../data/prop_65_batch_100')
pipeline(['#mata#',"#paga#",'#sai#','#bate#'])

#ma
Decodificado Loop: #ma, Final antes do Loop: tu#
#pa
Decodificado Loop: #pa, Final antes do Loop: gu#
#sa
Decodificado Loop: #sa, Final antes do Loop: iu#
#ba
Decodificado Loop: #ba, Final antes do Loop: pu#


(4, 460)

|     Model Name     | Test File     | Total Irregular Verbs | Total Verbs | Proportion | Epochs | Batch Size  |
|:------------------:|---------------|----------------------:|-------------|------------|--------|-------------|
| prop_65_batch_579_2       | prop_65       |            209        |  579        |    65%     | 1400   |  579         |

In [56]:
model = load_model('../data/prop_65_batch_579')
model.fit(X,Y, epochs=400, batch_size=40,verbose=False)
model.save('../data/prop_65_batch_579_2')

In [57]:
model = load_model('../data/prop_65_batch_579_2')
pipeline(['#pega#','#sega#','#seka#','#testa#','#presta#'])

#pe
Decodificado Loop: #pe, Final antes do Loop: tu#
#se
Decodificado Loop: #se, Final antes do Loop: gu#
#se
Decodificado Loop: #se, Final antes do Loop: ku#
#de
Decodificado Loop: #de, Final antes do Loop: tu#
#pr
Decodificado Loop: #pr, Final antes do Loop: tu#


(5, 460)

In [58]:
model = load_model('../data/prop_65_batch_579_2')
pipeline(['#ora#',"#mora#",'#joga#','#aposta#','#hetoma#'])

#ea
Decodificado Loop: #ea, Final antes do Loop: ru#
#be
Decodificado Loop: #be, Final antes do Loop: zu#
#xo
Decodificado Loop: #xo, Final antes do Loop: gu#
#de
Decodificado Loop: #de, Final antes do Loop: tu#
#xe
Decodificado Loop: #xe, Final antes do Loop: du#


(5, 460)

In [59]:
model = load_model('../data/prop_65_batch_579_2')
pipeline(['#sortia#','#media#'])

#to
Decodificado Loop: #to, Final antes do Loop: pu#
#be
Decodificado Loop: #be, Final antes do Loop: iu#


(2, 460)

In [60]:
model = load_model('../data/prop_65_batch_579_2')
pipeline(['#vense#','#eskolie#'])

#be
Decodificado Loop: #be, Final antes do Loop: su#
#bd
Decodificado Loop: #bd, Final antes do Loop: fu#


(2, 460)

In [96]:
model = load_model('../data/prop_65')
pipeline(['#po#','#hepo#'])

#po
Decodificado Loop: #po#, Final antes do Loop: iu#
#xe
Decodificado Loop: #xebuu#, Final antes do Loop: iu#


(2, 460)

In [97]:
model = load_model('../data/prop_65_1000')
pipeline(['#jenti#','#fenti#','#hendi#','#tendi#'])

#je
Decodificado Loop: #jentu, Final antes do Loop: tu#
#fe
Decodificado Loop: #fentu, Final antes do Loop: tu#
#xe
Decodificado Loop: #xentu#, Final antes do Loop: du#
#te
Decodificado Loop: #tentk#, Final antes do Loop: du#


(4, 460)

In [101]:
model = load_model('../data/prop_65_1000')
pipeline(['#tosi#','#kobri#'])#,'#fosi#']) 

#to
Decodificado Loop: #to#, Final antes do Loop: su#
#ko
Decodificado Loop: #kobsu#, Final antes do Loop: ru#


(2, 460)

In [102]:
model = load_model('../data/prop_65_1000')
pipeline(['#afaze#','#nase#'])

#af
Decodificado Loop: #afu#, Final antes do Loop: su#
#na
Decodificado Loop: #natu#, Final antes do Loop: su#


(2, 460)

In [103]:
model = load_model('../data/prop_65_1000')
pipeline(['#mata#',"#paga#",'#sai#','#bate#'])

#ma
Decodificado Loop: #matu, Final antes do Loop: tu#
#pa
Decodificado Loop: #pagu, Final antes do Loop: gu#
#sa
Decodificado Loop: #sau#, Final antes do Loop: iu#
#ba
Decodificado Loop: #batu#, Final antes do Loop: pu#


(4, 460)

## Proporcao 75% Irregulares

|     Model Name     | Test File     | Total Irregular Verbs | Total Verbs | Proportion | Epochs | Batch Size  |
|:------------------:|---------------|----------------------:|-------------|------------|--------|-------------|
| prop_75       | prop_75       |                      |  400        |    75%     | 600   |  20         |

In [106]:
model = load_model('../data/prop_75')
pipeline(['#pega#','#sega#','#seka#','#testa#','#presta#'])

#pe
Decodificado Loop: #pegu, Final antes do Loop: gu#
#si
Decodificado Loop: #sigu, Final antes do Loop: gu#
#si
Decodificado Loop: #siku, Final antes do Loop: ku#
#te
Decodificado Loop: #te#, Final antes do Loop: tu#
#pr
Decodificado Loop: #presps#, Final antes do Loop: tu#


(5, 460)

In [109]:
model = load_model('../data/prop_75')
pipeline(['#ora#',"#mora#",'#aposta#'])

#er
Decodificado Loop: #eru, Final antes do Loop: ru#
#mi
Decodificado Loop: #misu#, Final antes do Loop: ru#
#ap
Decodificado Loop: #apu#, Final antes do Loop: tu#


(3, 460)

In [110]:
model = load_model('../data/prop_75')
pipeline(['#sortia#','#media#'])

#su
Decodificado Loop: #su#, Final antes do Loop: iu#
#me
Decodificado Loop: #medu#, Final antes do Loop: iu#


(2, 460)

In [112]:
model = load_model('../data/prop_75')
pipeline(['#vense#'])

#ve
Decodificado Loop: #vensu, Final antes do Loop: su#


(1, 460)

In [114]:
model = load_model('../data/prop_75')
pipeline(['#po#','#hepo#','#kompo#'])

#po
Decodificado Loop: #pompp#, Final antes do Loop: iu#
#xe
Decodificado Loop: #xebuu#, Final antes do Loop: iu#
#ko
Decodificado Loop: #kompi#, Final antes do Loop: iu#


(3, 460)

In [116]:
model = load_model('../data/prop_75')
pipeline(['#fenti#','#hendi#','#tendi#'])

#fe
Decodificado Loop: #fentu, Final antes do Loop: tu#
#xe
Decodificado Loop: #xedtu, Final antes do Loop: tu#
#te
Decodificado Loop: #tentk#, Final antes do Loop: tu#


(3, 460)

In [118]:
model = load_model('../data/prop_75')
pipeline(['#kobri#'])#,'#fosi#']) 

#ko
Decodificado Loop: #komsu#, Final antes do Loop: ru#


(1, 460)

In [119]:
model = load_model('../data/prop_75')
pipeline(['#afaze#','#nase#'])

#ap
Decodificado Loop: #apu#, Final antes do Loop: su#
#na
Decodificado Loop: #naku#, Final antes do Loop: su#


(2, 460)

In [120]:
model = load_model('../data/prop_75')
pipeline(['#mata#',"#paga#",'#sai#','#bate#'])

#ma
Decodificado Loop: #matu, Final antes do Loop: tu#
#pa
Decodificado Loop: #pabu#, Final antes do Loop: gu#
#sa
Decodificado Loop: #sai#, Final antes do Loop: au#
#ba
Decodificado Loop: #batu, Final antes do Loop: tu#


(4, 460)

|     Model Name     | Test File     | Total Irregular Verbs | Total Verbs | Proportion | Epochs | Batch Size  |
|:------------------:|---------------|----------------------:|-------------|------------|--------|-------------|
| prop_75_1000       | prop_75       |                      |  400        |    75%     | 1000   |  20         |

In [121]:
model = load_model('../data/prop_75')
model.fit(X,Y, epochs=400, batch_size=20,verbose=False)
model.save('../data/prop_75_1000')

In [122]:
model = load_model('../data/prop_75_1000')
pipeline(['#pega#','#sega#','#seka#','#testa#','#presta#'])

#pe
Decodificado Loop: #pegu, Final antes do Loop: gu#
#si
Decodificado Loop: #sigu, Final antes do Loop: gu#
#si
Decodificado Loop: #sikO#, Final antes do Loop: ku#
#te
Decodificado Loop: #te#, Final antes do Loop: tu#
#pr
Decodificado Loop: #prespx#, Final antes do Loop: tu#


(5, 460)

In [127]:
model = load_model('../data/prop_75_1000')
pipeline(['#ora#',"#mora#",'#aposta#'])

#er
Decodificado Loop: #eru, Final antes do Loop: ru#
#mo
Decodificado Loop: #moza#, Final antes do Loop: ru#
#ap
Decodificado Loop: #apu#, Final antes do Loop: tu#


(3, 460)

In [128]:
model = load_model('../data/prop_75_1000')
pipeline(['#sortia#','#media#'])

#su
Decodificado Loop: #su#, Final antes do Loop: iu#
#me
Decodificado Loop: #meeo#, Final antes do Loop: iu#


(2, 460)

In [129]:
model = load_model('../data/prop_75_1000')
pipeline(['#vense#'])

#ve
Decodificado Loop: #vensu, Final antes do Loop: su#


(1, 460)

In [130]:
model = load_model('../data/prop_75_1000')
pipeline(['#po#','#hepo#','#kompo#'])

#po
Decodificado Loop: #po#, Final antes do Loop: iu#
#xe
Decodificado Loop: #xebaO#, Final antes do Loop: iu#
#ko
Decodificado Loop: #kompi#, Final antes do Loop: iu#


(3, 460)

In [131]:
model = load_model('../data/prop_75_1000')
pipeline(['#po#','#hepo#','#kompo#'])

#po
Decodificado Loop: #po#, Final antes do Loop: iu#
#xe
Decodificado Loop: #xebaO#, Final antes do Loop: iu#
#ko
Decodificado Loop: #kompi#, Final antes do Loop: iu#


(3, 460)

In [136]:
model = load_model('../data/prop_75_1000')
pipeline(['#fenti#','#hendi#'])

#fe
Decodificado Loop: #fentu, Final antes do Loop: tu#
#xe
Decodificado Loop: #xedtu, Final antes do Loop: tu#


(2, 460)

In [139]:
model = load_model('../data/prop_75_1000')
pipeline(['#kobri#'])#,'#fosi#']) 

#ko
Decodificado Loop: #komsu#, Final antes do Loop: ru#


(1, 460)

In [140]:
model = load_model('../data/prop_75_1000')
pipeline(['#afaze#','#nase#'])

#af
Decodificado Loop: #afu#, Final antes do Loop: su#
#na
Decodificado Loop: #napu#, Final antes do Loop: su#


(2, 460)

In [141]:
model = load_model('../data/prop_75_1000')
pipeline(['#mata#',"#paga#",'#sai#','#bate#'])

#ma
Decodificado Loop: #matu, Final antes do Loop: tu#
#pa
Decodificado Loop: #pabu#, Final antes do Loop: gu#
#sa
Decodificado Loop: #safpipipppp#, Final antes do Loop: au#
#ba
Decodificado Loop: #batu#, Final antes do Loop: pu#


(4, 460)

## Proporcao 85% Irregulares

|     Model Name     | Test File     | Total Irregular Verbs | Total Verbs | Proportion | Epochs | Batch Size  |
|:------------------:|---------------|----------------------:|-------------|------------|--------|-------------|
| prop_85       | prop_85       |                      |  400        |    85%     | 600   |  20         |

In [144]:
model = load_model('../data/prop_85')
pipeline(['#pega#','#sega#','#seka#','#testa#','#presta#'])

#pe
Decodificado Loop: #pegu, Final antes do Loop: gu#
#si
Decodificado Loop: #sigu, Final antes do Loop: gu#
#si
Decodificado Loop: #siku, Final antes do Loop: ku#
#de
Decodificado Loop: #de#, Final antes do Loop: tu#
#pr
Decodificado Loop: #presps#, Final antes do Loop: tu#


(5, 460)

In [147]:
model = load_model('../data/prop_85')
pipeline(['#ora#',"#mora#",'#joga#'])

#er
Decodificado Loop: #eru, Final antes do Loop: ru#
#mu
Decodificado Loop: #muzujfzazasakada#, Final antes do Loop: ru#
#jo
Decodificado Loop: #jogogu, Final antes do Loop: gu#


(3, 460)

In [148]:
model = load_model('../data/prop_85')
pipeline(['#sortia#','#media#'])

#su
Decodificado Loop: #su#, Final antes do Loop: iu#
#me
Decodificado Loop: #meuu#, Final antes do Loop: iu#


(2, 460)

## FULL BATCH

In [13]:
model = load_model('../data/prop_55_batch_400')
pipeline(['#pega#'])

#pe
Decodificado Loop: #pe, Final antes do Loop: gu#


(1, 460)